In [65]:
import pandas as pd
import numpy as np

In [2]:
import pickle

In [54]:
from operator import itemgetter

In [3]:
with open('gb_V2.pkl' , 'rb') as f:
    gb = pickle.load(f)

In [4]:
dfs = pd.read_html(r"https://en.wikipedia.org/wiki/2022_FIFA_World_Cup#Teams")

In [58]:
matches = []
groups = ["A", "B", "C", "D", "E", "F", "G", "H"]
group_count = 0 

table = {}
#TABLE -> TEAM, POINTS, WIN PROBS (CRITERIO DE DESEMPATE)
table[groups[group_count]] = [[a.split(" ")[0], 0, []] for a in list(dfs[11].iloc[:, 1].values)]

for i in range(12, 67, 1):
    if len(dfs[i].columns) == 3:
        team_1 = dfs[i].columns.values[0]
        team_2 = dfs[i].columns.values[-1]
        
        matches.append((groups[group_count], team_1, team_2))
    else:
        group_count+=1
        table[groups[group_count]] = [[a, 0, []] for a in list(dfs[i].iloc[:, 1].values)]

In [59]:
table

{'A': [['Qatar', 0, []],
  ['Ecuador', 0, []],
  ['Senegal', 0, []],
  ['Netherlands', 0, []]],
 'B': [['England', 0, []],
  ['Iran', 0, []],
  ['United States', 0, []],
  ['Wales', 0, []]],
 'C': [['Argentina', 0, []],
  ['Saudi Arabia', 0, []],
  ['Mexico', 0, []],
  ['Poland', 0, []]],
 'D': [['France', 0, []],
  ['Australia', 0, []],
  ['Denmark', 0, []],
  ['Tunisia', 0, []]],
 'E': [['Spain', 0, []],
  ['Costa Rica', 0, []],
  ['Germany', 0, []],
  ['Japan', 0, []]],
 'F': [['Belgium', 0, []],
  ['Canada', 0, []],
  ['Morocco', 0, []],
  ['Croatia', 0, []]],
 'G': [['Brazil', 0, []],
  ['Serbia', 0, []],
  ['Switzerland', 0, []],
  ['Cameroon', 0, []]],
 'H': [['Portugal', 0, []],
  ['Ghana', 0, []],
  ['Uruguay', 0, []],
  ['South Korea', 0, []]]}

In [60]:
matches[:5]

[('A', 'Qatar', 'Ecuador'),
 ('A', 'Senegal', 'Netherlands'),
 ('A', 'Qatar', 'Senegal'),
 ('A', 'Netherlands', 'Ecuador'),
 ('A', 'Ecuador', 'Senegal')]

In [13]:
base_df = pd.read_csv("full_db.csv")

In [14]:
base_df.columns

Index(['date', 'home_team', 'away_team', 'rank_home', 'rank_away',
       'home_score', 'away_score', 'result', 'rank_dif', 'points_dif',
       'rank_change_home', 'rank_change_away', 'home_goals_mean',
       'home_goals_mean_l5', 'home_goals_suf_mean', 'home_goals_suf_mean_l5',
       'home_rank_mean', 'home_rank_mean_l5', 'home_points_mean',
       'home_points_mean_l5', 'away_goals_mean', 'away_goals_mean_l5',
       'away_goals_suf_mean', 'away_goals_suf_mean_l5', 'away_rank_mean',
       'away_rank_mean_l5', 'away_points_mean', 'away_points_mean_l5',
       'home_game_points_mean', 'home_game_points_mean_l5',
       'home_game_points_rank_mean', 'home_game_points_rank_mean_l5',
       'away_game_points_mean', 'away_game_points_mean_l5',
       'away_game_points_rank_mean', 'away_game_points_rank_mean_l5',
       'is_friendly_0', 'is_friendly_1'],
      dtype='object')

In [15]:
def find_stats(team_1):
#team_1 = "Qatar"
    last_game = base_df[(base_df["home_team"] == team_1) | (base_df["away_team"] == team_1)].tail(1)

    if last_game["home_team"].values[0] == team_1:
        team_1_rank = last_game["rank_home"].values[0]
        team_1_goals = last_game["home_goals_mean"].values[0]
        team_1_goals_l5 = last_game["home_goals_mean_l5"].values[0]
        team_1_goals_suf = last_game["home_goals_suf_mean"].values[0]
        team_1_goals_suf_l5 = last_game["home_goals_suf_mean_l5"].values[0]
        team_1_rank_suf = last_game["home_rank_mean"].values[0]
        team_1_rank_suf_l5 = last_game["home_rank_mean_l5"].values[0]
        team_1_gp_rank = last_game["home_game_points_rank_mean"].values[0]
        team_1_gp_rank_l5 = last_game["home_game_points_rank_mean_l5"].values[0]
    else:
        team_1_rank = last_game["rank_away"].values[0]
        team_1_goals = last_game["away_goals_mean"].values[0]
        team_1_goals_l5 = last_game["away_goals_mean_l5"].values[0]
        team_1_goals_suf = last_game["away_goals_suf_mean"].values[0]
        team_1_goals_suf_l5 = last_game["away_goals_suf_mean_l5"].values[0]
        team_1_rank_suf = last_game["away_rank_mean"].values[0]
        team_1_rank_suf_l5 = last_game["away_rank_mean_l5"].values[0]
        team_1_gp_rank = last_game["away_game_points_rank_mean"].values[0]
        team_1_gp_rank_l5 = last_game["away_game_points_rank_mean_l5"].values[0]
    
    return [team_1_rank, team_1_goals, team_1_goals_l5, team_1_goals_suf, team_1_goals_suf_l5, team_1_rank_suf, team_1_rank_suf_l5, team_1_gp_rank, team_1_gp_rank_l5]

In [26]:
def find_features(team_1, team_2):
    rank_dif = team_1[0] - team_2[0]
    goals_dif = team_1[1] - team_2[1]
    goals_dif_l5 = team_1[2] - team_2[2]
    goals_suf_dif = team_1[3] - team_2[3]
    goals_suf_dif_l5 = team_1[4] - team_2[4]
    goals_per_ranking_dif = (team_1[1]/team_1[5]) - (team_2[1]/team_2[5])
    dif_rank_agst = team_1[5] - team_2[5]
    dif_rank_agst_l5 = team_1[6] - team_2[6]
    dif_gp_rank = team_1[7] - team_2[7]
    dif_gp_rank_l5 = team_1[8] - team_2[8]
    
    return [rank_dif, goals_dif, goals_dif_l5, goals_suf_dif, goals_suf_dif_l5, goals_per_ranking_dif, dif_rank_agst, dif_rank_agst_l5, dif_gp_rank, dif_gp_rank_l5, 1, 0]

In [88]:
advanced_group = []
last_group = ""
thresh = 0.05

for k in table.keys():
    for t in table[k]:
        t[1] = 0
        t[2] = []
        
for teams in matches:
    draw = False
    team_1 = find_stats(teams[1])
    team_2 = find_stats(teams[2])

    

    features = find_features(team_1, team_2)
    
    
In [120]:
￼
​
Out[120]:
[['Netherlands', 'United States'],
 ['Argentina', 'Denmark'],
In [120]:
￼
​
Out[120]:
[['Netherlands', 'United States'],
 ['Argentina', 'Denmark'],
    
    if (team_1_prob + thresh >= team_2_prob - thresh) & (team_1_prob < team_2_prob):
        draw=True
        for i in table[teams[0]]:
            if i[0] == teams[1] or i[0] == teams[2]:
                i[1] += 1
                
    elif team_1_prob >= team_2_prob:
        winner = teams[1]
        winner_proba = team_1_prob
        for i in table[teams[0]]:
            if i[0] == teams[1]:
                i[1] += 3
                
    elif team_1_prob + thresh < team_2_prob - thresh:  
        winner = teams[2]
        winner_proba = team_2_prob
        for i in table[teams[0]]:
            if i[0] == teams[2]:
                i[1] += 3
    
    for i in table[teams[0]]: #adding criterio de desempate (probs por jogo)
            if i[0] == teams[1]:
                i[2].append(team_1_prob)
            if i[0] == teams[2]:
                i[2].append(team_2_prob)

    if last_group != teams[0]:
        if last_group != "":
            print("\n")
            print("Group %s advanced: "%(last_group))
            
            for i in table[last_group]: #adding crieterio de desempate
                i[2] = np.mean(i[2])
            
            final_points = table[last_group]
            final_table = sorted(final_points, key=itemgetter(1, 2), reverse = True)
            advanced_group.append([final_table[0][0], final_table[1][0]])
            for i in final_table:
                print("%s -------- %d"%(i[0], i[1]))
        print("\n")
        print("-"*10+" Starting Analysis for Group %s "%(teams[0])+"-"*10)
        
        
    if draw == False:
        print("Group %s - %s vs. %s: Winner %s with %.2f probability"%(teams[0], teams[1], teams[2], winner, winner_proba))
    else:
        print("Group %s - %s vs. %s: Draw"%(teams[0], teams[1], teams[2]))
    last_group =  teams[0]

print("\n")
print("Group %s advanced: "%(last_group))

for i in table[last_group]: #adding crieterio de desempate
    i[2] = np.mean(i[2])
            
final_points = table[last_group]
final_table = sorted(final_points, key=itemgetter(1, 2), reverse = True)
advanced_group.append([final_table[0][0], final_table[1][0]])
for i in final_table:
    print("%s -------- %d"%(i[0], i[1]))



---------- Starting Analysis for Group A ----------
Group A - Qatar vs. Ecuador: Winner Ecuador with 0.67 probability
Group A - Senegal vs. Netherlands: Winner Netherlands with 0.65 probability
Group A - Qatar vs. Senegal: Winner Senegal with 0.62 probability
Group A - Netherlands vs. Ecuador: Winner Netherlands with 0.67 probability
Group A - Ecuador vs. Senegal: Draw
Group A - Netherlands vs. Qatar: Winner Netherlands with 0.75 probability


Group A advanced: 
Netherlands -------- 9
Senegal -------- 4
Ecuador -------- 4
Qatar -------- 0


---------- Starting Analysis for Group B ----------
Group B - England vs. Iran: Winner England with 0.52 probability
Group B - United States vs. Wales: Draw
Group B - Wales vs. Iran: Draw
Group B - England vs. United States: Winner England with 0.52 probability
Group B - Wales vs. England: Winner England with 0.77 probability
Group B - Iran vs. United States: Winner United States with 0.65 probability


Group B advanced: 
England -------- 9
United

In [89]:
advanced = advanced_group

In [82]:
playoffs = {"Round of 16": [], "Quarter-Final": [], "Semi-Final": [], "Final": []}

In [130]:
for p in playoffs.keys():
    playoffs[p] = []

actual_round = ""
next_rounds = []

for p in playoffs.keys():
    if p == "Round of 16":
        control = []
        for a in range(0, len(advanced*2), 1):
            if a < len(advanced):
                if a % 2 == 0:
                    control.append((advanced*2)[a][0])
                else:
                    control.append((advanced*2)[a][1])
            else:
                if a % 2 == 0:
                    control.append((advanced*2)[a][1])
                else:
                    control.append((advanced*2)[a][0])

        playoffs[p] = [[control[c], control[c+1]] for c in range(0, len(control)-1, 1) if c%2 == 0]
        
        for i in range(0, len(playoffs[p]), 1):
            game = playoffs[p][i]
            if i < len(advanced)/2:
                home = game[0]
                away = game[1]
                team_1 = find_stats(home)
                team_2 = find_stats(away)
            else:
                home = game[1]
                away = game[0]
                team_1 = find_stats(home)
                team_2 = find_stats(away)
            features = find_features(team_1, team_2)
            
            probs = gb.predict_proba([features])
            team_1_prob = probs[0][0]
            team_2_prob = probs[0][1]
            
            if actual_round != p:
                print("-"*10)
                print("Starting simulation of %s"%(p))
                print("-"*10)
                print("\n")
            
            if team_1_prob < team_2_prob:
                print("%s vs. %s: %s advances with prob %.2f"%(home, away, away, team_2_prob))
                next_rounds.append(away)
            else:
                print("%s vs. %s: %s advances with prob %.2f"%(home, away, home, team_1_prob))
                next_rounds.append(home)
            actual_round = p
        
    else:
        playoffs[p] = [[next_rounds[c], next_rounds[c+1]] for c in range(0, len(next_rounds)-1, 1) if c%2 == 0]
        next_rounds = []
        for game in playoffs[p]:
            home = game[0]
            away = game[1]
            team_1 = find_stats(home)
            team_2 = find_stats(away)
            features = find_features(team_1, team_2)
            
            probs = gb.predict_proba([features])
            team_1_prob = probs[0][0]
            team_2_prob = probs[0][1]
            
            if actual_round != p:
                print("-"*10)
                print("Starting simulation of %s"%(p))
                print("-"*10)
                print("\n")
            
            if team_1_prob < team_2_prob:
                print("%s vs. %s: %s advances with prob %.2f"%(home, away, away, team_2_prob))
                next_rounds.append(away)
            else:
                print("%s vs. %s: %s advances with prob %.2f"%(home, away, home, team_1_prob))
                next_rounds.append(home)
            actual_round = p

----------
Starting simulation of Round of 16
----------


Netherlands vs. United States: United States advances with prob 0.52
Argentina vs. Denmark: Argentina advances with prob 0.57
Spain vs. Croatia: Spain advances with prob 0.53
Brazil vs. Uruguay: Brazil advances with prob 0.55
England vs. Senegal: England advances with prob 0.55
France vs. Poland: France advances with prob 0.60
Belgium vs. Germany: Germany advances with prob 0.54
Portugal vs. Switzerland: Switzerland advances with prob 0.53
----------
Starting simulation of Quarter-Final
----------


United States vs. Argentina: Argentina advances with prob 0.73
Spain vs. Brazil: Brazil advances with prob 0.62
England vs. France: France advances with prob 0.58
Germany vs. Switzerland: Switzerland advances with prob 0.52
----------
Starting simulation of Semi-Final
----------


Argentina vs. Brazil: Brazil advances with prob 0.66
France vs. Switzerland: France advances with prob 0.53
----------
Starting simulation of Final
------

In [135]:
playoffs

{'Round of 16': [['Netherlands', 'United States'],
  ['Argentina', 'Denmark'],
  ['Spain', 'Croatia'],
  ['Brazil', 'Uruguay'],
  ['Senegal', 'England'],
  ['Poland', 'France'],
  ['Germany', 'Belgium'],
  ['Switzerland', 'Portugal']],
 'Quarter-Final': [['United States', 'Argentina'],
  ['Spain', 'Brazil'],
  ['England', 'France'],
  ['Germany', 'Switzerland']],
 'Semi-Final': [['Argentina', 'Brazil'], ['France', 'Switzerland']],
 'Final': [['Brazil', 'France']]}

In [134]:
find_features(find_stats("Brazil"), find_stats("France"))

[-3.0,
 0.056882327399045085,
 2.4,
 -0.4198871037776813,
 -0.8,
 0.0173815667174778,
 -10.858011289622233,
 20.799999999999997,
 -0.06742210342186801,
 0.0511841004901866,
 1,
 0]

[['Netherlands', 'United States'],
 ['Argentina', 'Denmark'],
 ['Spain', 'Croatia'],
 ['Brazil', 'Uruguay'],
 ['Senegal', 'England'],
 ['Poland', 'France'],
 ['Germany', 'Belgium'],
 ['Switzerland', 'Portugal']]